# 24. 맵리듀스로 그리는 빅데이터 지도

## 1. 들어가며

### 학습 목표
---
* 분산환경에서 빅데이터를 처리하는 기본 패턴을 이해한다.
* 맵리듀스의 기본 로직에 대해 이해한다.
* 맵리듀스가 적용된 하둡, 스파크의 개요를 알아본다.
* 함수형 프로그래밍을 통한 맵리듀스 실행 모델 구동을 실습해본다.

### 학습 내용
---
* 빅데이터의 기본패턴
* Split-Apply-Combine 전략
* 맵리듀스(MapReduce)
* 분산환경의 개념
* 하둡과 스파크
* 맵리듀스 실행 모델
* 파이썬 함수형 프로그래밍
* 맵리듀스 WordCount 예제 실습

## 2. '빅'데이터 처리의 기본패턴
제타바이트(Zettabyte, $10^{21}$ 단위까지 증가한 데이터, 즉 빅데이터를 처리하기 위한 방법은 뭐가 있을까? 말 그대로 데이터 용량도 크고 처리 시간도 많이 소요될 것이다. 그렇다면 우선 성능이 좋은 컴퓨터를 써야한다. 혹은 컴퓨터를 여러 대를 연결한 뒤 작업을 분할해 사용할 수도 있겠다. 요즘 컴퓨터는 멀티 코어, 멀티 프로세서라고 하니, 이런 컴퓨팅 자원을 엮어서 __병렬 컴퓨팅(Parallel Computing), 분산 컴퓨팅(Distributed Computing)__ 을 통해 단일 컴퓨터로는 처리는커녕 디스크에 담아둘 수조차 없을 만큼 큰 사이즈의 데이터로 다뤄 볼 수 있을 것이다. 비슷한 개념으로 __클러스터 컴퓨팅(Cluster Computing)__이라는 것도 있다. 오늘은 이런 개념들이 어떻게 다른지 공부할 것이다.

빅데이터라는 용어는 시장에서 다소 버즈 워드(buzz word) 취급을 받는 경향도 있다. 빅데이터에 내재된 활용하여 긴밀하게 전략을 구사하지 않으면 21세기의 경쟁 사회에서 뒤처질 수도 있다는 주장에 딱히 반대를 하지는 않지만, 과연 빅데이터란 무엇이고 어떻게 처리되는 것인지 정확한 메커니즘은 다소 혼란스럽다. 빅데이터를 다루는 솔루션이라는 하둡, 스파크 등의 생소한 이름들도 빅데이터 처리 메커니즘이란 어떤 것인지 한 꺼풀 들춰보기 다소 부담스럽게 한다.

오늘 다룰 __맵리듀스(Mapreduce)__라는 것은 하둡, 스파크 등 빅데이터를 다루는 솔루션들의 가장 근간이 되는 프로그래밍 모델이다. 약간은 추상적인 개념이라 바로 직관적으로 이해하기 어렵지만, 괜찮다. 내가 처리해야 할 데이터의 양이 매우 많다? 어떻게 해야 할까? 실은 뻔하다. __)많은 것을 잘게 나누어서 2)각각을 처리한 후에 3)처리된 것들을 모아서 통합 결과물을 낸다.__  그게 바로 맵리듀스 개념의 전부이다. 실은 아주 상식적인 개념이고, 우리도 일상에서 아주 많이 쓰고 있는 개념이기 때문에 적응하는데 어렵지 않을 것이다.

### 어느 만석꾼 부잣집 이야기
---
당신이 조선 시대 어느 만석꾼 부잣집 일꾼이라고 하자. 하루는 주인 양반이 이런 지시를 내린다. "이보게, 이 쌀 한 그릇에 모래가 들어갔네. 여기서 모래를 다 골라내게!"

어떻게 해야 할까? 한 그릇 분량의 쌀은 대충 2000 알갱이 정도 된다고 한다.(참고 - [통계청](https://m.post.naver.com/viewer/postView.nhn?volumeNo=10165636&memberNo=608322) ) 매초 한 알씩 쉬지 않고 열심히 쌀 알갱이와 모래를 골라냈다. 그래서 30여분이 지나서 '주인님 다 되었습니다!' 하고 보고할 수 있게 되었다.

그랬더니, 주인 양반 왈, "아이구 참 잘했네. 그런데 보니 이 쌀을 퍼낸 저 가마니 안에 온통 이렇게 모래가 들어가 있지 뭔가, 이보게, 이거 전부 다 좀 골라내 주게. 이거 다 골라내지 않으면 오늘 밥은 없네."<br>
큰일 났다. 쌀 한가마니는 대략 4000그릇이라고 한다. 아까 방법으로 모래를 골라냈다가는… 대략 2400시간, 딱 100일 동안 밥을 못 먹게 될 것이다.

이 소식을 들은 동네의 친구 100명이 몰려들었다. 이 착하고 성실한 친구들은 하루종일 쉬지도 않고 열심히 여러분과 같은 방법으로 모래를 골라서 깨끗한 쌀알만을 추려 모아 다시 깨끗한 쌀 한가마니를 만들어 냈다<br>.
이렇게 맵리듀스 마을의 친구들은 멋진 분산처리를 통해 당신을 위기에서 구해낼 수 있었다. 그들이 쓴 방법은 바로,

* 1) 한가마니의 쌀을 100등분으로 __잘게 나누어서__
* 2) 한 명당 그 1/100을 가져가 __각각 처리__(한알씩 골라내기)를 열심히 한 후에
* 3) 각자가 분리해 낸 깨끗한 쌀을 __한데 모아__ 다시 한가마니를 만들어 냈다.

바로 정확히 맵리듀스의 개념과 일치한다.

### 빅데이터 처리의 기본 패턴
---
일꾼 혼자서는 처리할 수 없는 일거리를 여럿이 나눠서 처리하는 것과, 컴퓨터 한 대로는 처리할 수 없는 분량의 데이터를 여러 대가 나눠서 처리하는 작업의 원리는 같다. 이것을 데이터분석의 __Split-Apply-Combine Strategy__라고 한다. Mapreduce는 이런 모델의 한 특수한 형태라고 할 수 있다. (참고 -[위키피디아](https://en.wikipedia.org/wiki/MapReduce))

## 3. Split-Apply-Combine
아래는 `Split-Apply-Combine Strategy`에 대해 명쾌하게 설명해 주고 있는 글이다.

[Split-Apply-Combine Strategy for Data Mining](https://medium.com/analytics-vidhya/split-apply-combine-strategy-for-data-mining-4fd6e2a0cc99)

* 위에 부잣집 일꾼 예제에서 한 가마니의 쌀을 100등분으로 나누었다. 나누기 위해 특별히 적용한 기준은 없고, 동일한 크기로 나누어 각각의 친구에게 할당했다. 1/100가마니로 나뉜 모래 섞인 쌀이 GroupBy object가 된다. 이게 Split 단계(GoupBy object 생성)이다.
* Split된 object를 2개의 카테고리(쌀, 모래)로 분리하는 function이 apply되므로, Filter라고 할 수 있다. Apply 단계로 적용된 function인 것이다.

윗글에 따르면 1)많은 것을 잘게 나누어서__(Split)__ 2)각각을 처리한 후에__(Apply)__ 3)처리된 것들을 모아서 통합 결과물을 낸다__(Combine)__ 는 전략은 SQL의 GroupBy 문, R의 plyr 함수 등 다양한 곳에 적용되고 있다. 오늘 살펴볼 Mapreduce 도 그런 좋은 사례가 될 것이다.

그럼 Mapreduce에 대해 위의 관점으로 좀 더 구체적으로 들여다보자. 아래 링크는 2004년 구글의 제프리 딘(Jeffrey Dean)과 산제이 기마와트(Sanjay Ghemawat)가 발표한 _MapReduce: Simplified Data Processing on Large Clusters_ 란 논문이다. 이 논문이 오늘날 빅데이터용 클러스터 컴퓨팅에 사용되는 주요 프로그래밍 모델인 맵리듀스를 최초로 제안한 공식 자료이다.

[MapReduce: Simplified Data Processing on Large Clusters](https://static.googleusercontent.com/media/research.google.com/ko//archive/mapreduce-osdi04.pdf)

아래 그림은 이 논문의 Mapreduce 개념도를 바탕으로 이전 스텝의 일꾼 비유를 Split-Apply-Combine Strategy 관점으로 재구성한 것이다.

<img src="./image/mapreduce.png" alt="MapReduce" />

Mapreduce는 크게 `map()` 함수와 `reduce()` 함수로 구성되어 있다. 그래서 합쳐서 Mapreducej이다.

* `map()` 함수는 Split된 부분 데이터를 가져다가 어떤 특별한 조작을 가하는, `Apply` 역할을 하는 함수이다. 일꾼 비유에서 본다면, 모래섞인 쌀을 가져다가 한알 한알 쌀과 모래로 분리하고 있는 친구가 이 `map()` 함수 역할을 하고 있다.
* `reduce()` 함수는 `map()` 함수가 만들어낸 결과물을 어떤 기준에 따라 한군데로 다시 모아내는, `Combine` 역할을 하는 함수이다. 일꾼 비유에서 본다면, 친구들이 분리해 놓은 쌀과 모래들을 가져다가 쌀만 한 가마니에 모아담는 일꾼이 `reduce()` 함수 역할을 하고 있다.

## 4. 맵리듀스의 원리
맵리듀스는 하나의 컴퓨터에서 하던 작업을 여러 개의 컴퓨터에서 처리하도록 분산시키는 프로그래밍 모델이다. 이 모델은 처리하고자 하는 작업을 여러 컴퓨터에 자동으로 분할하고 지역 처리 및 클러스터 노드 간 통신을 하기 위해 사용한다.

### 맵리듀스 논문
---
2004년 구글의 제프리 딘(Jeffrey Dean)과 산제이 기마와트(Sanjay Ghemawat)는 _MapReduce: Simplified Data Processing on Large Clusters__ 란 논문을 발표했다.

이 논문이 오늘날 빅데이터용 클러스터 컴퓨팅에 사용되는 주요 프로그래밍 모델인 맵리듀스를 최초로 제안한 공식 자료이다. 아래 링크에 가면 논문의 원문을 확인할 수 있다. 처음 접하는 사람에게는 조금 어려운 내용일 수도 있다. 논문의 
* Abstract 
* 2.1 Types 
* Figure 1: Execution overview 
* 8. Conclusions 

단락만 읽어도 괜찮다.

* [MapReduce: Simplified Data Processing on Large Clusters](https://static.googleusercontent.com/media/research.google.com/ko//archive/mapreduce-osdi04.pdf)

그리고 추가로 도움이 될만한 좋은 자료가 있다. 논문의 저자들이 작성한 요약 설명 슬라이드 자료이다.

* [MapReduce: Simplified Data Processing on Large Clusters - Google Research](https://research.google.com/archive/mapreduce-osdi04-slides/index.html)

아래 그림은 이 슬라이드의 3페이지에 언급된 Mapreduce의 프로그래밍 모델 개요이다.

<img src="./image/outline.png" alt="MapReduce" />

Mapreduce는 `map`와 `reduce` 2개의 함수만 강조되고 있는 것 같지만, 이것 또한 실은 Split 단계를 전제로 하고 있다. `map` 함수의 인자로 `in_key`를 받는 것을 보면 알 수 있다.

위 그림에서 `map` 함수를 보면 `in_key`와 `in_value`를 인자로 갖는다. `in_key`란 바로 Split의 결과로 생긴 partitioning 키값이다. 일꾼 예제에서라면 0번~99번 친구의 키값이라고 할 수 있겠다. 이 키값은 최종 Output에는 반영되지 않는다. 어떤 쌀알을 구분한 사람이 몇 번 친구인지는 중요하지 않기 때문이다.

`map`함수는 Split된 데이터를 가져다가 `out_key`와 `intermediate_value`의 리스트로 변환한다. out_key는 map함수가 결과물을 구분하는 기준 키값이다. 일꾼 예제라면 0:쌀, 1:모래 같은 형태가 되겠다. 그래서 일꾼 예제의 map 친구들은 길이 2인 list에 각각 깨끗한 쌀과 모래를 담아서 리턴할 것이다.

`reduce` 함수의 입력은 여러 `map`함수의 `intermediate_value`들을 out_key별로 구분해서 리스트업한 것을 입력으로 한다. 일꾼 예제라면 100명의 친구들이 모아 준 쌀 100덩어리와 모래 100덩어리가 입력되겠다. 이것을 가지고 `recuce` 함수는 `out_key` 기준으로 sum을 할 것이다. 그래서 `reduce` 함수인 일꾼은 최종적으로 `out_value`의 list를 출력한다. 즉, 전체 쌀과 전체 모래를 모아놓은 길이 2의 list가 최종 출력이 될 것이다.

<img src="./image/execution.png" alt="MapReduce Execution" />

위 그림은 Mapreduce의 실행(Execution) 모델이다. M이 Input을 입력받아 Intermediate로, R이 Intermediate를 `Out_key`기준으로 Grouping한 것을 입력으로 받아 Output을 출력하는 구조로 진행되는 것을 볼 수 있다.

여기서 초기 Mapreduce의 한가지 약점이라고도 할수 있는 특징을 알 수 있다. `reduce` 함수는 모든 `map` 함수가 중간 결과물을 리턴할 때까지 기다렸다가 한꺼번에 그룹핑한 결과가 나올 때까지 기다린다. 일꾼 예제를 예로 든다면, 어느 한 친구의 작업이 많이 늦어져서 쌀과 모래가 전부 취합되지 않는다면 우리의 일꾼은 마지막 한 친구의 작업이 끝날 때까지 한없이 기다려야 한다는 것이다. (뭔가 다른 일을 할 수도 있었을 텐데...) 이런 시스템이 가지고 있는 잠재적 약점이 있다.

이 Execution model 그림에서 보면 `map` 함수뿐 아니라 `reduce` 함수도 병렬수행 가능하다는 것을 알 수 있다. 일꾼 예제에서 `reduce` 함수 역할을 일꾼 혼자서 하는 걸로 하였지만 실제로는 `reduce` 함수도 여러 개를 병렬수행 가능하다.

## 5. 분산환경의 빅데이터 처리
맵리듀스에 대한 대략적인 감이 잡혔다. 이제 맵리듀스를 제대로 배우려면 먼저 어떤 개념을 배워야 할지, 조금 더 쉽게 설명되어 있는 사이트에서 관련 내용을 찾아보자.

* [Wikipedia: MapReduce](https://en.wikipedia.org/wiki/MapReduce)

위키피디아에 언급된 설명과 관련 기술을 아래 그림과 같이 지도로 표현해 보겠다.

<img src="./image/wiki.png" alt="MapReduce" />

우선 __맵리듀스의 정의__와 관련된 개념이 나온다.

* 클러스터 위에 동작하는 알고리즘
* 프로그래밍 모델이다.

그리고 __맵리듀스의 로직__을 소개한다.

* Map, Reduce , Shuffle

또… 눈에 띄지 않을 수 있는데, __함수형 프로그래밍(Functional Programming)__ 역시 매우 중요한 개념이다.

위 지도를 따라서 맵리듀스를 지탱하는 기술에는 어떤 개념들이 있는지 하나씩 따라가 보자.

### 병렬, 분산, 클러스터 컴퓨팅
---
첫 번째 스텝에서, 컴퓨터 한 대로 감당할 수 없는 빅데이터를 처리하기 위해서는 병렬컴퓨팅이나 분산컴퓨팅이 필요하다고 했다. 이 두 가지 개념을 혼용해서 사용하는 경우가 많지만, 엄밀하게는 다음과 같이 다른 개념이다.

#### 병렬 컴퓨팅(Parallel Computing)
이 개념은 한 대의 컴퓨터 안에서 CPU 코어를 여러 개 사용해서 한 대의 컴퓨터가 처리하는 데이터의 총량과 처리속도를 증가시키자는 것이다. 관련하여 아래와 같은 개념들이 중요하게 다뤄진다.

* 멀티 프로세스 (Multiprocessing)
    - 프로세스는 컴퓨터에서 연속적으로 실행되고 있는 컴퓨터 프로그램을 의미한다. 스케줄링의 대상이 되는 작업(task)이라는 용어로도 쓰인다. 프로세스 관리는 OS에서 한다.
    - 멀티 프로세스는 2개 이상의 프로세스를 사용하는 것이다.
* 멀티 스레드 (Multithreading)
    - 스레드(thread)는 어떠한 프로그램 내에서, 특히 프로세스 내에서 실행되는 흐름의 단위이다. 일반적으로 한 프로그램은 하나의 스레드를 가진다.
    - 멀티 스레드는 2개 이상의 스레드를 사용하는 것이다.

#### 분산 컴퓨팅
이 개념은 여러 대의 컴퓨터가 네트워크로 연결된 상황을 전제로 하는 것이다. 따라서 아래와 같은 네트워크 용어들이 중요하게 다뤄진다.

* P2P(Peer to Peer)
* HTTP
* Network

#### 클러스터 컴퓨팅
위 그림을 보면 클러스터 컴퓨팅이 위의 병렬 컴퓨팅과 분산 컴퓨팅을 통합하는 개념처럼 제시되고 있다. 실제로 이 용어는 분산 컴퓨팅과 매우 혼용되고 있기도 하다.

* __여러 대의 컴퓨터들이 연결되어 하나의 시스템처럼 동작__하는 컴퓨터들의 집합이다.
* 보통 노드와 관리자로 구성되어 있다.
    - 클러스터 노드 : 프로세싱 자원을 제공하는 시스템이다.
    - 클러스터 관리자 : 노드를 서로 연결하여 단일 시스템처럼 보이게 만드는 로직을 제공하다.
* 대표적인 기술
    - 병렬 컴퓨팅(Parellel Computing)
    - 분산 컴퓨팅(Distributed Computing)
    - 클라우드 컴퓨팅(Cloud Computing)
    
클러스터는 여러 대의 컴퓨터들이 연결되어 있는 것이다. 이때 컴퓨터를 어떻게 연결하고 컴퓨팅 자원을 관리할 것인지에 따라서 많은 클러스터 관련 기술이 알려져 있다.
다음 스텝에서 소개될 하둡, 스파크 같은 시스템들은 이런 클러스터 컴퓨팅을 전제로 설계된 시스템이다.

응용의 주안점에 따라 위에 언급한 병렬 컴퓨팅, 분산 컴퓨팅, 클라우드 컴퓨팅도 있고, 클러스터에서 어떻게 데이터를 교환할 것인지에 대한 부분, 그리고 데이터 서버를 클러스터로 관리한다면 파일 시스템을 어떻게 연결할지에 대한 부분 등 다양한 분야에서 연구가 이루어지고 있다. 분야가 방대하지만 지금은 기본적으로 클러스터란 여러 대의 컴퓨터를 묶여서 하나의 시스템처럼 사용하는 것이라는 것만 이해하면 된다.

## 6. 하둡과 스파크

<img src="./image/hadoop.jpg" alt="hadoop" />

하둡은 대용량 데이터를 분산 처리할 수 있는 자바 기반의 오픈소스 프레임워크이다. 2004년에 구글에서 Mapreduce가 발표되고, 여기에 GFS(Google File System)이라는 구글의 분산 파일 시스템이 나오면서 이미 천문학적인 크기가 된 인터넷 검색 데이터를 분산처리하는 기술을 통해 구글이 기술적 우위를 공고히 하게 된다. 특히 GFS는 구글에서 공개를 하지 않았다. 이에 아파치 재단(Apache Foundation)에서는 구글의 맵리듀스 및 GFS에 해당하는 HDFS(Hadoop File system)를 포함하는 하둡(Hadoop)이라는 오픈소스 프로젝트를 2006년 4월에 발표하게 된다. 이후 하둡은 오픈소스라는 강점을 내세워 대략의 데이터 처리를 지원하는 사실상의 표준 프레임워크로 군림하게 된다.

<img src="./image/spark.png" alt="spark" />

그러나, 하둡이 가지는 약점이 있었다. 이것은 문제는 Mapreduce의 문제이기도 하다. map함수가 모두 종료해야 reduce함수가 실행된다는 가져오는 성능 손실이 컸기 때문이다. 이것이 사실상 하둡으로 실시간 서비스를 제공하는 것을 불가능하게 했고, 하둡은 빅데이터 기반의 배치성 통계작업에만 주로 투입되고 있다.<br>
반면 2009년에 처음 발표된 스파크(Spark)는 종래의 Mapreduce 기능에서 Map함수가 전부 종료되지 않았더라도 Map의 결과를 스트리밍하는 방식으로, Map의 결과가 다 나와야만 Reduce를 수행한다는 전제를 깨버리게 된다.

<img src="./image/execution2.png" alt="execution" />

그림을 보면 한가지 설계적 문제가 눈에 띈다. 그것은 바로 map 함수의 결과가 디스크에 저장되고, 그걸 reduce 함수가 다시 읽어와야 한다는 점이다. 이것이 주는 성능상의 손실이 크다. 반면 스파크는 인메모리 데이터 엔진을 통해 초기 맵리듀스의 성능상 한계를 크게 극복하게 된다. 실제로 Spark의 성능은 Hadoop의 10배에 달하는 속도 우위를 보인다.

그 외에도 정말 다양한 스파크의 강점이 있다. 그러나 주의할 점은, 그렇다고 스파크가 하둡에 절대 우위를 보인다는 것은 아니다. 하둡은 스파크가 제공하지 않는 HDFS같은 분산 파일 시스템을 제공하고 있다. 스파크 자체가 하둡 기반으로 구동하는 것을 목적으로 만들어졌기 때문에, 꼭 하둡이 필요한 것은 아니지만 하둡 기반으로 돌아가는 것을 추천한다.

[ITWorld:"하둡을 제압한 빅데이터 플랫폼" 아파치 스파크란 무엇인가](https://www.itworld.co.kr/insight/147556)

간과할 수 없는 스파크의 또다른 특징 중에는, 자바(Java), 스칼라(Scala), 파이썬(Python), R 프로그래밍 언어를 위한 네이티브 바인딩을 제공해서, 다양한 언어가 필요한 환경에서 손쉽고 빠르게 스파크와 연동할 수 있다는 점이 있다. 그래서 서버사이드에서는 자바나 스칼라를 이용해 스파크를 구성하고, 데이터 분석가들은 친숙한 파이썬을 이용해 데이터를 처리하는 경우가 많다.

게다가 분석가들을 위해서 SQL, 스트리밍 데이터, 머신러닝, 그래프 프로세싱을 추가로 지원한다. 스파크를 사용하게 되면, 마치 내 컴퓨터에 있는 csv파일을 Pandas로 읽어 DataFrame으로 분석하는 것처럼, 내 컴퓨터에는 저장도 하기 힘들만큼의 데이터를 DataFrame으로 처리할 수 있다는 장점이 있다. 사용자 입장에서는 스파크 뒤에 있는 수많은 컴퓨터들에 분산저장되어 있는 거대한 데이터가 전부 내가 쓰고 있는 컴퓨터 한 대에 들어 있는 것처럼 착각하게 만드는 클러스터 컴퓨팅을 기반으로 하는 것이다. 그리고, 이 모든 강력한 기능을 파이썬을 통해 다룰 수 있다는 것이 무엇보다도 두드러진 장점일 것이다.

## 7. 함수, 파이썬의 1급 시민
파이썬을 이용한 맵리듀스 프로그래밍에 앞서 언급해야 할 중요한 내용이 있다.

바로 파이썬에서 함수가 1급 시민이라는 점이다. 이 특징은 함수형 프로그래밍 개념으로 파이썬에서 맵리듀스를 활용하는 것을 매우 강력하게 한다.

1급 시민이라니.. 뭔가 특별한 대우를 받을 것만 같다. 맞다. 파이썬에서는 함수가 매우 특별하다. 그리고 함수가 1급 시민이기 때문에 파이썬에서는 함수형 프로그래밍이 가능한 것이다.

"1급 시민"이란 말은 함수가 다음과 같은 기능을 할 수 있음을 의미한다.

* 함수는 다른 함수의 인자로 전달될 수 있다. (= 인자(매개변수)로 전달이 가능)
* 함수는 변수에 할당될 수 있다.
* 함수는 다른 함수의 결과로서 반환될 수 있다. (=다른 함수의 반환값)

### 함수는 다른 함수의 인자로 전달될 수 있다.
---
여기 2개의 함수가 있다.

In [1]:
#에러메세지를 출력하는 함수
def errormessage(format_func, msg):
    print(format_func(msg))

#단어의 첫글자만 대문자로 만드는 함수
def to_upper(msg):
    t = msg.split(" ")
    T = []
    for i in t:
        i = i[0].upper() + i[1:]
        T.append(i)
    T = " ".join(T)
    return T

`errormessage()` 함수의 인자로 `character()` 함수를 전달할 수 있다. 아래 같은 경우 텍스트를 다르게 변형하는 함수를 만들어 (모든 글자를 대문자로 만들어주는 함수는 어떨까?) 전달하면 텍스트가 또 다른 형태로 포맷팅 될 것이다.

In [2]:
msg = "you have limited access"
errormessage(to_upper, msg)

You Have Limited Access


### 함수는 변수에 할당될 수 있다.
---
매개변수로 전달될 수 있다는 의미는 변수로써 할당이 가능하다는 뜻이다. 함수를 키워드 인자에 할당할 수 있죠. 이것이 가능한 이유는 파이썬에서 함수가 1급 시민이기 때문이다.

In [3]:
def france():
    print('bonjour')

def italy():
    print('ciao')

위는 평범한 함수이다. 이제 `hello`란 변수에 `france()` 함수를 할당한다. 파이썬에서는 가능합니다.

In [4]:
hello = france
print(hello)

<function france at 0x7f64705e84d0>


그러면 `hello`는 함수처럼 사용이 가능하다. 그리고 `hello`를 출력하면 `france()` 함수라고 나온다. 마지막으로 `hello`의 자료형은` function` 클래스, 즉 1급 시민이다.

In [5]:
hello()

bonjour


In [6]:
print(type(hello))

<class 'function'>


### 함수는 다른 함수의 결과로서 반환될 수 있다.
---
마지막 특징은 함수의 `return`값으로 함수를 사용할 수 있다는 특징이다.

In [7]:
def func1(x):
    return x**2

def func2():
    return func1

In [8]:
print(func1(3))
print(func2()(3))

9
9


파이썬에서 "모든 것이 객체다"라는 말 들어보았을 것이다. 그중에서도 으뜸은 함수이다.

함수형 프로그래밍은 모든 것을 객체로 표현하게끔 하는 것이다. 모순적인 말 같지만 함수 또한 객체기 때문에 함수를 값으로 할당하고 다른 함수의 결괏값으로 반환할 수 있다.

다른 프로그래밍 언어는 어떨까?

* 1급 시민인 언어 : Javascript, Scala, Go 등
* 함수가 1급 시민이 아닌 언어 : C, Java 등

>__*함수형 프로그래밍 철학*__
>
> 1. *변경 가능한 상태를 불변의 상태(Immutab)로 만들어 에러를 없애자.*
> 2. *모든 것은 객체이다.*
> 3. *코드를 간결하게 하고 가독성을 높여 구현할 로직에 집중한다.*
> 4. *보다 효율적인 동시성 작업*

## 8. 함수형 프로그래밍과 맵리듀스

### 데이터 컬렉션
---
파이썬 자료 구조 중에 List, Tuple, Dictionary, Set와 같은 형태의 자료형을 컬렉션(Collection) 이라고 하기도 한다. 다른 말로는 컨테이너(Container)라고 부른다.

In [9]:
import collections

파이썬은 `collections` 컬렉션 모듈을 통해서 위 자료형 외 데이터 컬렉션 타입을 제공하기도 한다.

`collections` 모듈에서 제공하는 데이터 컬렉션은 다음과 같다.

<img src="./image/collections.png" alt="Collection" />

여기서 `deque`, `namedtuple()`, `defaultdict` 은 자주 사용된다.

* [파이썬 공식문서: collections - Container datatypes - Python 3.8.3rc1 documentation](https://docs.python.org/3.8/library/collections.html)
컬렉션을 이야기한 이유는 빅데이터라는 대용량의 자료를 표현할 때 이러한 타입으로 저장하는 경우가 매우 많기 때문이다. 위에 나온 클래스가 그대로 쓰이지는 않지만, 키와 값 등 기본적인 철학을 공유하는 경우가 많으니 그 원리를 이해하는게 좋다.

아래에 `defaultdict` 자료구조를 이용하여 다음 문장에 쓰인 단어의 개수를 세는 코드를 구현해보자. (단어는 공백을 기준으로 구분되며, don't과 같은 축약형 단어는 한 단어로 취급한다.)

In [10]:
from collections import defaultdict

text = """I have a depression, and then there was a girl who came into 
my life. One day, my life was changed because that girl just changed my 
life. She taught me how to love and how to be an active person. 
And then, I feel so happy when I am always with her. 
I love her so much. I don't want let her go. I am sad because she 
is with her favorite friends. I will do anything to make her proud"""

result = defaultdict(int)
word = text.split()

for i in word:
    result[i] += 1
    
result   # 결과 출력

defaultdict(int,
            {'I': 7,
             'have': 1,
             'a': 2,
             'depression,': 1,
             'and': 2,
             'then': 1,
             'there': 1,
             'was': 2,
             'girl': 2,
             'who': 1,
             'came': 1,
             'into': 1,
             'my': 3,
             'life.': 2,
             'One': 1,
             'day,': 1,
             'life': 1,
             'changed': 2,
             'because': 2,
             'that': 1,
             'just': 1,
             'She': 1,
             'taught': 1,
             'me': 1,
             'how': 2,
             'to': 3,
             'love': 2,
             'be': 1,
             'an': 1,
             'active': 1,
             'person.': 1,
             'And': 1,
             'then,': 1,
             'feel': 1,
             'so': 2,
             'happy': 1,
             'when': 1,
             'am': 2,
             'always': 1,
             'with': 2,
             'her.': 1,


### 맵리듀스 기본 함수 알아보기
---
파이썬에서 함수를 데이터 컬렉션에 적용하는 패턴에는 3가지가 있다.

* map
* filter
* reduce
파이썬에 있는 함수들을 사용해 비슷한 연산을 해보며 맵리듀스의 원리를 공부할 것이다. map 연산을 통해 여러 개의 컴퓨터에 연산을 나누고, reduce를 통해 여러 개 컴퓨터에 분산하여 연산한 결과를 합치면 원하는 결과를 빠르게 얻을 수 있다. 아래에서는 각 연산이 어떤 연산인지 간략히 감을 잡아보려 한다.

#### map(매핑)

<img src="./image/map.png" alt="map" />

컬렉션의 모든 요소에 함수를 적용(=매핑)합니다. 함수 $M(x)$를 컬렉션 $C$의 항목에 매핑하는 함수를 수식으로 나타내면 아래와 같다.

$$M(x):x\in C$$

문자열이 들어있는 데이터 컬렉션 `list` 에 `int` 함수를 적용시켜보자.

그리고 파이썬 람다(lambda) 함수를 적용하는 사례로 추가해보자.

In [11]:
mynum = ['1','2','3','4']
mynum_int = list(map(int, mynum))  # mynum의 각 원소에 int 함수를 적용
print(mynum_int)

mynum_square = list(map(lambda x : x*x, mynum_int))  # mynum_int의 각 원소 x에 lambda x : x*x 함수를 적용
print(mynum_square)

[1, 2, 3, 4]
[1, 4, 9, 16]


#### filter(필터링)

<img src="./image/filter.png" alt="filter" />

컬렉션 내의 요소를 선택(=필터링)한다. 컬렉션 $C$의 요소 $x$에 대해서 조건 $F(x)$에 부합하는 $x$만을 필터링하는 수식은 다음과 같다.

$$x:x\in C,\,  ifF(x)$$

주어진 리스트에서 양수인 요소만 필터링하는 코드이다.

In [12]:
mynum = range(-5, 5)  
mynum_plus = list(filter(lambda x: x > 0, mynum)) # mynum의 각 원소 x에 대해 lambda x: x > 0 인지 체크하는 필터를 적용
print(mynum_plus)

[1, 2, 3, 4]


#### reduce(리듀싱)

<img src="./image/reduce.png" alt="reduce" />

reduce(리듀싱)은 컬렉션을 축약한다. 가장 일반적인 예로는 요소들을 합하는 시그마 연산이 있다. 아래 수식은 컬렉션 $C$의 요소 $x$의 합을 구해서 축약한다.

$$\sum_{x\in C} x$$

코드로 확인해보자.

In [13]:
from functools import reduce
mynum = [1, 2, 3, 4, 5]
add = reduce((lambda x, y: x + y), mynum)  # reduce는 내부에 관리하는 x 변수에 mynum의 각 원소 y를 차례차례 더하여 x에 반영한다.

print(add)

15


Mapreduce는 map과 reduce만으로 이루어져 있는 보편적인 프로그래밍 모델이다. 이 모델이 보편성을 유지하려면 map이 적용되는 방식과 reduce가 적용되는 방식이 미리 정해져 있으면 안 된다. 위와 같이 map과 reduce가 적용되는 방식을 규정한 __함수를 인자로 받는__ 함수형 프로그래밍이 가능하다는 것은 Mapreduce 프로그래밍 모델을 매우 유연하고 강력하게 만들어 주게 된다.

## 9. 맵리듀스 예제
그럼 이제 실전 예제를 통해 맵리듀스를 구현해 보도록 하겠다.

### 주요 개념
맵리듀스는 4가지 단계가 있다. __Split - Map - Shuffle - Reduce__. Input과 Output은 각각의 키-값 쌍으로 존재한다고 했다. 각 단계별 입력값과 출력값을 살펴보자.

<img src="./image/input_output.png" alt="input_output" />

그리고 이 각각의 단계는 크게 __Map Task__와 __Reduce Task__로 구분할 수 있다.

<img src="./image/map_task.png" alt="map_task" />

### 예제 - 알파벳 수 찾기
---
알파벳 빈도수 찾기를 맵리듀스를 이용해 구현해보자.

<img src="./image/alphabet.png" alt="alphabet" />

각각 단계별로 함수를 수행했을 때 입력과 출력이 어떻게 될지 미리 살펴봅시다.

__1. 입력 데이터__

* hello python

__2. Split__

* h,e,l,l,o,(공백),p,y,t,h,o,n

__3. Map__

* (h:1, e:1, l:1, l:1, o:1, ' ':1, …)

__4. Reduce__

* (h:1, e:1, l:2…)

### 구현
---
__Mapper(매퍼): 입력데이터를 나누고 매핑한다.__

In [14]:
#input data
text = 'hello python'

def mapper(text):
    split = []
    for i in text:
        split.append((i, 1))
    return split

__Reducer: 매퍼에서 입력받은 값 중 같은 항목은 합친다.__

In [15]:
def reducer(split):
    out = {}
    for i in split:
        if i[0] not in out.keys():
            out[i[0]] = 1
        else:
            out[i[0]] += 1
    return out


__MapReduce: 출력은 다음과 같다.__

In [16]:
reducer(mapper(text))

{'h': 2, 'e': 1, 'l': 2, 'o': 2, ' ': 1, 'p': 1, 'y': 1, 't': 1, 'n': 1}

구글처럼 실제 데이터를 사용하여 대량의 데이터를 가공하게 되면 매퍼가 더 복잡해질 것이고, 컴퓨터를 여러 대 연결하거나 컴퓨터 자원을 최적화하여 계산하게 될 것이다.